In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-community langchain-openai tavily-python

In [54]:
import getpass
import os
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict
from langchain import hub
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from typing import Union
from typing import Literal
from langgraph.graph import END
from langgraph.graph import StateGraph, START

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

In [55]:
tools = [TavilySearchResults(max_results=3)]

In [56]:
prompt = hub.pull("ih/ih-react-agent-executor")
prompt.pretty_print()

llm = ChatOpenAI(model="gpt-4")
agent_executor = create_react_agent(llm, tools, state_modifier=prompt)

C:\Users\ArhamShams\anaconda3\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are a helpful assistant.

============================= Messages Placeholder =============================

{messages}


In [57]:
class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

## Planning Step

In [58]:
class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

In [59]:
planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Given a list of skills generate a detailed and unique contexts for each skillset. Your output will be used to create quiz questions \
            Each context should provide comprehensive and unique information about the skill, including its definition, applications, significance, and any notable sub-skills or components. Ensure the context is clear, precise, and factual.You are tasked to come up with a simple step by step plan on how you can create a context \
            This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
            The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ("placeholder", "{skillset}"),
    ]
)
planner = planner_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Plan)

In [60]:
planner.invoke(
    {
        "skillset": [
            ("user", "{Jira}")
        ]
    }
)

Plan(steps=['Research the definition of Jira and its primary purpose.', 'Identify the key features and functionalities of Jira.', 'Explore the various applications and industries where Jira is used.', 'Understand the significance of Jira in project management and software development.', 'Identify any notable sub-skills or components related to Jira, such as Jira Query Language (JQL) or integration capabilities.', 'Compile the gathered information into a comprehensive and unique context for Jira.'])

Re-Plan Step

In [61]:
class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """Given a list of skills generate a detailed and unique contexts for each skillset. Your output will be used to create quiz questions \
            Each context should provide comprehensive and unique information about the skill, including its definition, applications, significance, and any notable sub-skills or components. Ensure the context is clear, precise, and factual.You are tasked to come up with a simple step by step plan on how you can create a context \
            This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
            The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps. \

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)


replanner = replanner_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Act)

## Create the Graph

In [62]:
async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    agent_response = await agent_executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"skillset": [("user", state["input"])]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

In [63]:
workflow = StateGraph(PlanExecute)

workflow.add_node("planner", plan_step)

workflow.add_node("agent", execute_step)

workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

workflow.add_edge("planner", "agent")

workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    should_end,
    ["agent", END],
)

app = workflow.compile()

In [64]:
config = {"recursion_limit": 50}
inputs = {"input": "{Angular}"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

{'plan': ['Research the definition of Angular, including its history and development.', 'Identify the primary applications of Angular in web development.', 'Explore the significance of Angular in the context of modern web technologies.', 'List and describe notable sub-skills or components of Angular, such as modules, components, services, and directives.', 'Compile the information into a comprehensive context that includes definition, applications, significance, and sub-skills.']}
{'past_steps': [('Research the definition of Angular, including its history and development.', "AngularJS is a web development framework that was first developed in 2008-2009 by Miško Hevery and Adam Abrons at a company called Brat Tech LLC. Their original intent was to provide software for an online JSON storage service and simplify the development of enterprise applications measured by the megabyte. \n\nIn 2010, it was released as an open-source project under the maintenance of Google's team of engineers. S